# Prepping Attendance Data for Tableau Progress Monitoring Dashboard

The purpose of this script is to prepare a long format data file (.csv) to be used when updating weekly attendance in the KIPP Colorado Progress Monitoring Dashboard (SY21-22)

Datasets: 
- raw_attendance_last.csv: Average daily attendance from IC for the previous week (dates will change each week)
- raw_attendance_current.csv: Average daily attendance from IC for the current week (dates will each week)
- raw_studemo.csv: Student Demographics for Attendance ad-hoc file from IC (should be updated each week)

Project Start Date: 6/14/21

Most Recented Updated: 6/15/21


## Import Libraries and Read in .csv Files

In [1]:
import pandas as pd

In [2]:
current_week_attendance = pd.read_csv('raw_attendance_current.csv', 
                                      index_col = False, 
                                      usecols = ['School', 'Grade', 'Student Number', 'Percent In Attendance'],
                                                 skiprows=range(1,38))

last_week_attendance = pd.read_csv('raw_attendance_last.csv', 
                                      index_col = False, 
                                      usecols = ['School', 'Grade', 'Student Number', 'Percent In Attendance'],
                                                 skiprows=range(1,38))

stu_demo = pd.read_csv('raw_studemo.csv')

In [3]:
current_week_attendance.head()

,School,Grade,Student Number,Percent In Attendance
0,KIPP Denver Collegiate High School,09,773709,43.00%
1,KIPP Denver Collegiate High School,09,732128,86.80%
2,KIPP Denver Collegiate High School,09,733921,98.00%
3,KIPP Denver Collegiate High School,09,759098,52.40%
4,KIPP Denver Collegiate High School,09,740845,86.80%


In [4]:
last_week_attendance.head()

,School,Grade,Student Number,Percent In Attendance
0,KIPP Denver Collegiate High School,09,773709,100.00%
1,KIPP Denver Collegiate High School,09,732128,63.40%
2,KIPP Denver Collegiate High School,09,733921,95.60%
3,KIPP Denver Collegiate High School,09,759098,75.75%
4,KIPP Denver Collegiate High School,09,740845,100.00%


In [5]:
stu_demo.head()

,Import Student ID,Gender,Race Code 1,Primary Disability Code,English Proficiency/Language Fluency Code
0,629622,M,4,NaN,3: Redesignated ELL
1,654120,F,4,04: Specific Learning Disability,1: Non-ELL
2,647645,M,4,13: Autism Spectrum Disorders,5: Exit
3,632138,M,3,NaN,1: Non-ELL
4,780758,M,4,04: Specific Learning Disability,2: ELL


## Column Cleanup

- Replace the values for specific columns (race, MLL, SWD)
- Change datatype for percent attendance
- Rename headers for Tableau analysis

In [6]:
#Create dictionaries to replace the codes from Infinite Campus
race_codes = {
    1: 'American Indian or Alaskan Native',
    2: 'Asian',
    3: 'Black',
    4: 'Hispanic',
    5: 'White',
    6: 'Native Hawaiian or Other Pacific Islander',
    7: 'Multiple Races'
}

swd_codes = {'04: Specific Learning Disability': 'SWD',
             '13: Autism Spectrum Disorders': 'SWD',
             '03: Serious Emotional Disability': 'SWD',
             '16: Other Health Impairment': 'SWD',
             '14: Traumatic Brain Injury': 'SWD',
             '01: Intellectual Disability': 'SWD',
             '05: Hearing Impairment, Including Deafness': 'SWD',
             '08: Speech or Language Impairment': 'SWD',
             '11: Developmental Delay': 'SWD',
             '10: Multiple Disabilities': 'SWD',
             '06: Visual Impairment, Including Blindness': 'SWD'
             }

#Assume that only students with a 2:ELL or 4: Provisional ELL are coded as MLL
mll_codes = {'0: Missing/Invalid HLQ': 'Non-MLL',
             '1: Non-ELL': 'Non-MLL',
             '2: ELL': 'MLL',
             '3: Redesignated ELL': 'Non-MLL',
             '4: Provisional ELL': 'MLL',
             '5: Exit': 'Non-MLL'
             }

In [7]:
#Replace all codes from Infinite Campus
stu_demo['Race Code 1'].replace(race_codes, inplace = True)
stu_demo['Primary Disability Code'].replace(swd_codes, inplace = True)
stu_demo['English Proficiency/Language Fluency Code'].replace(mll_codes, inplace = True)

In [8]:
#Replace the blank values with Non-SWD for primary disability and Non-MLL for ELL code
stu_demo['Primary Disability Code'].fillna('Non-SWD', inplace = True)
stu_demo['English Proficiency/Language Fluency Code'].fillna('Non-MLL', inplace = True)

In [9]:
current_week_attendance['Percent In Attendance'] = current_week_attendance['Percent In Attendance'].str.rstrip('%').astype('float') / 100.0

last_week_attendance['Percent In Attendance'] = last_week_attendance['Percent In Attendance'].str.rstrip('%').astype('float') / 100.0

In [10]:
current_week_attendance.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2625 entries, 0 to 2624
Data columns (total 4 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   School                 2625 non-null   object 
 1   Grade                  2625 non-null   object 
 2   Student Number         2625 non-null   int64  
 3   Percent In Attendance  2625 non-null   float64
dtypes: float64(1), int64(1), object(2)
memory usage: 82.2+ KB


In [11]:
# Replace the headers
stu_demo.rename(columns={
    'Import Student ID': 'Student ID',
    'Race Code 1': 'Race',
    'Primary Disability Code': 'SWD',
    'English Proficiency/Language Fluency Code': 'MLL'
}, inplace=True)

current_week_attendance.rename(columns={
    'Student Number': 'Student ID',
    'Percent In Attendance': 'Current Week Attendance'
}, inplace=True)

last_week_attendance.rename(columns={
    'Student Number': 'Student ID',
    'Percent In Attendance': 'Last Week Attendance'
}, inplace=True)

In [12]:
current_week_attendance.head()

,School,Grade,Student ID,Current Week Attendance
0,KIPP Denver Collegiate High School,09,773709,0.430
1,KIPP Denver Collegiate High School,09,732128,0.868
2,KIPP Denver Collegiate High School,09,733921,0.980
3,KIPP Denver Collegiate High School,09,759098,0.524
4,KIPP Denver Collegiate High School,09,740845,0.868


## Join Tables & Determine Attendance Difference

- Join the previous attendance to current attendance on an inner join
- Create a column to calculate the difference in attendance by week
- Join the demographics to the attendance

In [13]:
attendance = pd.merge(current_week_attendance, last_week_attendance, 
                      on = ['School', 'Grade', 'Student ID'],
                      how = 'outer')

attendance.head()

,School,Grade,Student ID,Current Week Attendance,Last Week Attendance
0,KIPP Denver Collegiate High School,09,773709,0.430,1.0000
1,KIPP Denver Collegiate High School,09,732128,0.868,0.6340
2,KIPP Denver Collegiate High School,09,733921,0.980,0.9560
3,KIPP Denver Collegiate High School,09,759098,0.524,0.7575
4,KIPP Denver Collegiate High School,09,740845,0.868,1.0000


In [14]:
attendance['Change in Attendance'] = attendance['Current Week Attendance'] - attendance['Last Week Attendance']

In [15]:
complete_attendance = pd.merge(attendance, stu_demo,
                              on = 'Student ID',
                              how = 'outer')

complete_attendance.head()

,School,Grade,Student ID,Current Week Attendance,Last Week Attendance,Change in Attendance,Gender,Race,SWD,MLL
0,KIPP Denver Collegiate High School,09,773709,0.430,1.0000,-0.5700,M,Hispanic,Non-SWD,Non-MLL
1,KIPP Denver Collegiate High School,09,732128,0.868,0.6340,0.2340,M,Hispanic,Non-SWD,Non-MLL
2,KIPP Denver Collegiate High School,09,733921,0.980,0.9560,0.0240,M,Hispanic,Non-SWD,MLL
3,KIPP Denver Collegiate High School,09,759098,0.524,0.7575,-0.2335,M,Hispanic,Non-SWD,Non-MLL
4,KIPP Denver Collegiate High School,09,740845,0.868,1.0000,-0.1320,M,Hispanic,Non-SWD,Non-MLL


## Melt the Dataframe

- Use the demographics info as identifier variables and melt the attendance columns

In [16]:
tableau_attendance = pd.melt(complete_attendance, 
                             id_vars=['Student ID', 'School', 'Grade', 'Gender', 'Race','SWD', 'MLL', 'Change in Attendance'],
                             value_vars=['Current Week Attendance', 'Last Week Attendance'],
                             var_name='Attendance', 
                             value_name='Week').sort_values(by='Student ID')

In [17]:
tableau_attendance.head()

,Student ID,School,Grade,Gender,Race,SWD,MLL,Change in Attendance,Attendance,Week
3293,618339,KIPP Denver Collegiate High School,12,F,Hispanic,Non-SWD,Non-MLL,-0.080,Last Week Attendance,1.000
416,618339,KIPP Denver Collegiate High School,12,F,Hispanic,Non-SWD,Non-MLL,-0.080,Current Week Attendance,0.920
417,618340,KIPP Denver Collegiate High School,12,M,Hispanic,Non-SWD,Non-MLL,-0.080,Current Week Attendance,0.876
3294,618340,KIPP Denver Collegiate High School,12,M,Hispanic,Non-SWD,Non-MLL,-0.080,Last Week Attendance,0.956
3367,629622,KIPP Denver Collegiate High School,12,M,Hispanic,Non-SWD,Non-MLL,-0.192,Last Week Attendance,0.892


## Export the Dataframe


In [18]:
tableau_attendance.to_csv('tableau_ready_attendance.csv', index = False)